In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.onnx
import pandas as pd
from matplotlib import pyplot as plt
# matplotlib.use('Qt5Agg')
%matplotlib Qt5Agg
import livenet


In [32]:
train_x, train_y = livenet.datasets.get_odd_2()
network = livenet.nets.create_livenet_odd_2()


In [33]:

res = network(train_x)
network.context.regularization_l1 = 0.05
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.backend.optimizers.optimizers.LiveNetOptimizer(network, lr=0.01)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=50)
trainer.clear_life_stat = False



In [42]:
trainer.step(233)


Iˈ0.000 499 0.106+0.467reg params=9 av_abs_grad 0.03 av_abs_delta: 0.004 av_abs_output 1 max_abs_output 3 max_abs_param 2.8 hcut 0 lcut 0.2 livenet/net_trainer.py:121
Iˈ0.167 549 0.106+0.467reg params=9 av_abs_grad 0.03 av_abs_delta: 0.004 av_abs_output 1 max_abs_output 3 max_abs_param 2.9 hcut 0 lcut 0.2 livenet/net_trainer.py:121
Iˈ0.288 599 0.106+0.467reg params=9 av_abs_grad 0.03 av_abs_delta: 0.004 av_abs_output 1 max_abs_output 3 max_abs_param 3.0 hcut 0 lcut 0.2 livenet/net_trainer.py:121
Iˈ0.406 649 0.106+0.467reg params=8 av_abs_grad 0.03 av_abs_delta: 0.004 av_abs_output 1 max_abs_output 3 max_abs_param 3.1 hcut 0 lcut 0.2 livenet/net_trainer.py:121
Iˈ0.519 699 0.105+0.467reg params=8 av_abs_grad 0.03 av_abs_delta: 0.004 av_abs_output 2 max_abs_output 3 max_abs_param 3.1 hcut 0 lcut 0.2 livenet/net_trainer.py:121


In [36]:

trainer.step(1)


In [43]:
#livenet.utils.export_onnx(network, "/home/spometun/table/home/model.onnx")
print(f"total: {len(list(network.named_parameters()))}")
print(network.context.topology_stat.get_stat())
def name_comp_key(x: str):
    if "->" in x:
        x = "_" + x
    return x

for name, p in sorted(network.named_parameters(), key=lambda x: name_comp_key(x[0])):
    print(name, f"{p.item():.2f}")


total: 8
{'dangle': {'RegularNeuron': 0, 'DestinationNeuron': 1}, 'useless': {'RegularNeuron': 0, 'InputNeuron': 0}}
D0 -0.85
D1 1.31
N0 -2.65
N1 1.54
I0->N0 2.65
I0->N1 -1.56
N0->D1 -2.02
N1->D1 -3.10


In [45]:
life_stat = network.context.life_stat
df = pd.DataFrame(life_stat)
df2 = df[(df["type"] == "parameter") & (df["name"] == "N1->D0")]
#df2 = df[(df["type"] == "output_max") & (df["name"] == "N1->D0")]
df2.set_index("tick")
df2["value"].plot(grid=True, x="tick", y="value", style='.-', use_index=False)


<Axes: >

In [11]:
values = df2["value"].to_list()
ticks = list(df2["tick"])

plt.ion()
plt.figure(figsize=(16, 9))
plt.grid()
plt.plot(y=values, x=ticks)
# plt.show()

TypeError: plot() got an unexpected keyword argument 'x'

In [16]:
print(df2["tick"].head())

61     0
119    1
177    2
235    3
293    4
Name: tick, dtype: int64
